In [ ]:
import requests

result = requests.post("http://localhost:5173/trace", json={"a": 1, "b": 2})

In [ ]:
import json
import requests

trace = json.loads(
    '{"entries":[{"entries":[],"name":"test","start_timestamp":"2024-03-07T14:38:34.431530Z","end_timestamp":null,"trace_id":"7287488b-07c4-4340-b475-e5d50300a8c1","input":"input","output":null},{"entries":[],"name":"test","start_timestamp":"2024-03-07T14:38:43.064119Z","end_timestamp":null,"trace_id":"a188d64a-a7d3-4b4b-8c73-245c09d1a511","input":"input","output":null},{"entries":[],"name":"test","start_timestamp":"2024-03-07T14:38:52.839496Z","end_timestamp":null,"trace_id":"08561a77-41b6-4f3d-a052-8b36269203cc","input":"input","output":null}]}'
)


result = requests.post("http://localhost:5173/trace", json=trace)

In [ ]:
result.content